In [ ]:
# Step 1: Install YOLOv8 from Ultralytics
# pip install ultralytics

from ultralytics import YOLO
import os

# Step 2: Set Paths for Dataset
# Ensure your dataset follows the YOLO directory structure:
# ├── dataset/
# │   ├── images/
# │   │   ├── train/
# │   │   ├── val/
# │   ├── labels/
# │   │   ├── train/
# │   │   ├── val/
# │   ├── dataset.yaml

dataset_path = "dataset/data.yaml"  # Update with your dataset's path

# Step 3: Load Pretrained YOLOv8 Model
model = YOLO('yolov8n.pt')  # Choose 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', etc., based on your compute capacity

# Step 4: Train the Model
results = model.train(
    data=dataset_path,       # Path to the dataset YAML file
    epochs=5,              # Number of epochs (increase for better results)
    batch=16,                # Batch size (adjust based on GPU memory)
    imgsz=1280,               # Image size (higher can increase accuracy but requires more memory)
    workers=4,               # Number of data loading workers
    optimizer='AdamW',       # Use AdamW optimizer for better convergence
    lr0=0.001,               # Initial learning rate
    patience=10,             # Early stopping patience
    augment=True,            # Apply data augmentation (random scaling, flipping, etc.)
    val=True                 # Validate after training
)

# Step 5: Evaluate on the Validation Set
print("Evaluating on Validation Set:")
val_metrics = model.val()  # Automatically uses 'val' split from data.yaml

# Step 7: Inference on Custom Images
inference_results = model.predict(source="dataset/images/test", save=True, conf=0.25)
print("Inference completed. Check results in the 'runs/' folder.")

